In [1]:
from stravalib import Client
import jupyter_black
from datetime import datetime, timedelta
import pandas
from stravalib import Client
import altair as alt
import numpy

jupyter_black.load()

MY_STRAVA_CLIENT_ID = 125463
MY_STRAVA_CLIENT_SECRET = "017fafbf5c5067490f1382fd3454c30cdb61f4b0"
DAYS_BACK = 180

In [2]:
def authorize() -> Client:
    client = Client()
    url = client.authorization_url(
        client_id=125463, redirect_uri="http://127.0.0.1:5000/authorization"
    )

    print(f"Go to {url}")
    response = input(f"Enter the full URL:")
    code = response.split("code=")[1].split("&")[0]
    token_response = client.exchange_code_for_token(
        client_id=MY_STRAVA_CLIENT_ID, client_secret=MY_STRAVA_CLIENT_SECRET, code=code
    )
    access_token = token_response["access_token"]
    refresh_token = token_response["refresh_token"]

    return Client(access_token=access_token)


# client = Client()
# token_response = client.refresh_access_token(client_id=MY_STRAVA_CLIENT_ID,
#                                       client_secret=MY_STRAVA_CLIENT_SECRET,
#                                       refresh_token=last_refresh_token)
# new_access_token = token_response['access_token']

client = authorize()

Go to https://www.strava.com/oauth/authorize?client_id=125463&redirect_uri=http%3A%2F%2F127.0.0.1%3A5000%2Fauthorization&approval_prompt=auto&scope=read%2Cactivity%3Aread&response_type=code


In [3]:
five_months_ago = datetime.now() - timedelta(days=DAYS_BACK)
activities = client.get_activities(after=five_months_ago.isoformat())
running_activities = [act for act in activities if act.type == "Run"]

In [16]:
data = {
    "start_date": [act.start_date for act in running_activities],
    "distance_meters": [float(act.distance) for act in running_activities],
    "time_seconds": [act.moving_time.total_seconds() for act in running_activities],
}


df = pandas.DataFrame(data)
df["start_date"] = pandas.to_datetime(df["start_date"])
df.set_index("start_date", inplace=True)
weekly_data = df.resample("W").sum()
weekly_data["distance_km"] = round(weekly_data["distance_meters"] / 1000, 1)


def get_tanda_value(km_per_week: int, pace_sec_per_km: int) -> float:
    marathon_distance = 42.195
    marathon_pace_sec_per_km = (
        17.1 + 140.0 * numpy.exp(-0.0053 * km_per_week) + 0.55 * pace_sec_per_km
    )
    total_marathon_time_secs = marathon_distance * marathon_pace_sec_per_km
    total_marathon_time_hours = total_marathon_time_secs / 3600
    return total_marathon_time_hours


def get_pace_for_distance(km_per_week: int, total_marathon_time_hours: float) -> float:
    marathon_distance = 42.195
    marathon_pace_sec_per_km = total_marathon_time_hours * 3600 / marathon_distance
    pace_sec_per_km = (
        marathon_pace_sec_per_km - 17.1 - 140.0 * numpy.exp(-0.0053 * km_per_week)
    ) / 0.55
    return pace_sec_per_km


def pretty_marathon_time(total_marathon_time_hours: float) -> str:
    hours = int(total_marathon_time_hours)
    minutes = int((total_marathon_time_hours - hours) * 60)
    seconds = int(((total_marathon_time_hours - hours) * 60 - minutes) * 60)
    if seconds >= 30:
        minutes += 1

    return f"{hours} hours {minutes} minutes"

In [17]:
daily_df = df.groupby(df.index.date).sum()
daily_df.index = pandas.to_datetime(daily_df.index)
daily_df.index.name = "date"


daily_df["tanda_day"] = get_tanda_value(
    daily_df["distance_meters"] / 1000 * 7,
    daily_df["time_seconds"] / (daily_df["distance_meters"] / 1000),
)
daily_df["tanda_day_pretty"] = pandas.to_datetime(daily_df["tanda_day"], unit="h")


daily_df = daily_df.reset_index()
daily_df["date"] = pandas.to_datetime(daily_df["date"])
daily_df.set_index("date", inplace=True)

num_weeks = 8
num_days = num_weeks * 7
rolling = f"{num_days}d"


daily_df["rolling_distance_meters"] = (
    daily_df["distance_meters"].rolling(window=rolling).sum()
)
daily_df["rolling_time_seconds"] = (
    daily_df["time_seconds"].rolling(window=rolling).sum()
)

daily_df["rolling_km_per_week"] = daily_df["rolling_distance_meters"] / 1000 / num_weeks
daily_df["rolling_pace_sec_per_km"] = (
    daily_df["rolling_time_seconds"] / daily_df["rolling_distance_meters"] * 1000
)

daily_df["rolling_tanda_day"] = get_tanda_value(
    daily_df["rolling_km_per_week"], daily_df["rolling_pace_sec_per_km"]
)

daily_df["rolling_tanda_day_pretty"] = pandas.to_datetime(
    daily_df["rolling_tanda_day"], unit="h"
)

daily_df["type_rolling"] = "Tanda (8 weeks)"
daily_df["type_daily"] = "Tanda (daily)"

daily_df["pace_sec_per_km"] = daily_df["time_seconds"] / (
    daily_df["distance_meters"] / 1000
)
daily_df["distance_km"] = daily_df["distance_meters"] / 1000

daily_df = daily_df.sort_values(by="date", ascending=True)
daily_df["date_factor"] = numpy.exp(numpy.linspace(0, 15, len(daily_df)))


daily_df["daily_pace_pretty"] = daily_df["pace_sec_per_km"].apply(
    lambda x: f"{int(x//60)}:{int(x%60):02d}"
)
daily_df["rolling_pace_pretty"] = daily_df["rolling_pace_sec_per_km"].apply(
    lambda x: f"{int(x//60)}:{int(x%60):02d}"
)

daily_df["rolling_km_per_week_daily_distance"] = daily_df["rolling_km_per_week"] / 7
daily_df["Latest run"] = "Latest run"

daily_df["pretty_rolling_tanda_day"] = daily_df["rolling_tanda_day"].apply(
    pretty_marathon_time
)

# current_form_marathon_time = pretty_marathon_time(
#     daily_df.loc[start_date:last_date]
#     .reset_index()
#     .sort_values("date")
#     .tail(1)["rolling_tanda_day"]
#     .item()
# )


def pace_tick_formatter(value):
    minutes = int(value // 60)
    seconds = int(value % 60)
    return f"{minutes}:{seconds:02d}"

In [18]:
x_scale = alt.Scale(padding=20)
upper_limit = weekly_data["distance_km"].max()
lower_limit = 0

x = alt.X("start_date:T", scale=alt.Scale(padding=20), title="Week")
y = alt.Y(
    "distance_km:Q",
    axis=alt.Axis(title="Kilometers"),
    scale=alt.Scale(domain=[lower_limit, upper_limit]),
)

line_chart = (
    alt.Chart(weekly_data.reset_index())
    .mark_area(
        line={"color": "#ff561b"},
        color=alt.Gradient(
            gradient="linear",
            stops=[
                alt.GradientStop(color="white", offset=0),
                alt.GradientStop(color="#ff561b", offset=1),
            ],
            x1=1,
            x2=1,
            y1=1,
            y2=0,
        ),
    )
    .encode(
        x=x,
        y=y,
        tooltip=["start_date:T", "distance_km:Q"],
    )
    .properties(width=800, height=500, title="Running distance per week (km)")
)

points = (
    alt.Chart(weekly_data.reset_index())
    .mark_point(
        filled=True,
        fill="white",
        stroke="#ff561b",
        strokeWidth=2,
        size=50,
        shape="circle",
    )
    .encode(
        x=x,
        y=y,
        tooltip=["start_date:T", "distance_km:Q"],
    )
)


chart = line_chart + points

chart.show()

alt.LayerChart(...)

In [19]:
x = alt.X("date:T", title="Date", scale=alt.Scale(padding=20))

daily_line = (
    alt.Chart(daily_df.reset_index())
    .mark_point(shape="square", filled=True, opacity=0.5)
    .encode(
        x=x,
        y=alt.Y("hoursminutes(tanda_day_pretty):O", title="Tanda day"),
        color=alt.value("#d65de0"),
        tooltip=[
            alt.Tooltip("tanda_day_pretty", timeUnit="hoursminutes"),
            alt.Tooltip("date", timeUnit="yearmonthdate"),
        ],
    )
)
rolling_line = (
    alt.Chart(daily_df.reset_index())
    .mark_line(interpolate="basis")
    .encode(
        x=x,
        y=alt.Y(
            "hoursminutes(rolling_tanda_day_pretty):O", title="Tanda trend (8 weeks)"
        ),
        color=alt.value("#d65de0"),
        tooltip=[
            alt.Tooltip("rolling_tanda_day_pretty", timeUnit="hoursminutes"),
            alt.Tooltip("date", timeUnit="yearmonthdate"),
        ],
    )
)


(daily_line + rolling_line).properties(width=800, height=500, title="Tanda").show()

alt.LayerChart(...)

In [20]:
pace_ticks_values = list(range(240, 60 * 8, 15))
last_date = max(daily_df.index)
start_date = last_date - timedelta(days=56)


daily_df["shape"] = daily_df.index.to_series().apply(
    lambda x: "square" if x == last_date else "circle"
)


daily_line = (
    alt.Chart(daily_df.loc[start_date:last_date].reset_index().sort_values("date"))
    .mark_point(
        filled=True,
        # shape="triangle",
        size=90,
    )
    .encode(
        x=alt.X(
            "distance_km:Q",
            title="Daily Distance (km)",
            axis=alt.Axis(tickCount=int(25 // 5)),
        ),
        y=alt.Y(
            "pace_sec_per_km:Q",
            scale=alt.Scale(
                reverse=True,
                zero=False,
                domain=(min(pace_ticks_values), max(pace_ticks_values)),
            ),
            title="Pace (mm:ss)",
            axis=alt.Axis(
                values=pace_ticks_values,
                labelExpr="datum.value > 0 ? timeFormat(datum.value * 1000, '%M:%S') : ''",
            ),
        ),
        tooltip=[
            alt.Tooltip("distance_km:Q", title="Distance (km)", format=".1f"),
            alt.Tooltip("date:T", title="Date"),
            alt.Tooltip("daily_pace_pretty:N", title="Pace (mm:ss/km)"),
        ],
        color=alt.Color(
            "date_factor:Q", scale=alt.Scale(scheme="lightgreyred"), legend=None
        ),
        shape=alt.Shape(
            "shape:N", scale=alt.Scale(range=["square", "circle"]), legend=None
        ),
    )
)


(daily_line).properties(
    width=800, height=500, title="Pace and daily distance"
).interactive().show()

alt.Chart(...)

In [21]:
marathon_times = []


for marathon_time in numpy.arange(2.5, 4.5, 0.25):
    for km_day in range(0, 50, 1):
        km_week = km_day * 7
        pace = get_pace_for_distance(km_week, marathon_time)
        formatted_pace = pace_tick_formatter(pace)
        marathon_times.append(
            {
                "marathon_time": marathon_time,
                "km_day": km_day,
                "km_week": km_week,
                "pace": pace,
                "formatted_pace": formatted_pace,
            }
        )

times_df = pandas.DataFrame(marathon_times)

In [22]:
daily_df.loc[start_date:last_date].distance_km.max()

np.float64(14.549299999999999)

In [23]:
marathon_times = (
    alt.Chart(times_df)
    .mark_line(interpolate="basis")
    .encode(
        x=alt.X(
            "km_day:Q",
            title="Daily Distance (km)",
            scale=alt.Scale(
                domain=[0, daily_df.loc[start_date:last_date].distance_km.max()]
            ),
        ),
        y=alt.Y(
            "pace:Q",
            title="Pace (mm:ss)",
            scale=alt.Scale(
                reverse=True,
                zero=False,
                domain=(300, 405),
            ),
            axis=alt.Axis(
                values=pace_ticks_values,
                labelExpr="datum.value > 0 ? timeFormat(datum.value * 1000, '%M:%S') : ''",
            ),
        ),
        color=alt.Color(
            "marathon_time:N",
            title="Marathon Time",
            scale=alt.Scale(scheme="turbo"),
            legend=alt.Legend(
                labelExpr="floor(datum.value) + ':' + (floor((datum.value % 1) * 60) < 10 ? '0' : '') + floor((datum.value % 1) * 60)"
            ),
        ),
        tooltip=[
            alt.Tooltip("km_day:Q", title="Distance (km)"),
            alt.Tooltip("formatted_pace:N", title="Pace (mm:ss)"),
            alt.Tooltip("marathon_time:Q", title="Marathon time (hours)"),
        ],
    )
    .properties(width=800, height=500, title="Pace and daily distance")
    .interactive()
)

marathon_times.show()

alt.Chart(...)

In [24]:
daily_df["Legend"] = "Tanda Progression line"


tooltip = [
    alt.Tooltip(
        "rolling_km_per_week_daily_distance:Q",
        title="Distance (km)",
        format=".1f",
    ),
    alt.Tooltip(
        "rolling_pace_pretty:N",
        title="Pace (s/km)",
    ),
    alt.Tooltip("date:T", title="Date"),
    alt.Tooltip("pretty_rolling_tanda_day:N", title="Marathon Form"),
]


tanda_progression = (
    alt.Chart(daily_df.loc[start_date:last_date].reset_index().sort_values("date"))
    .mark_line(point=True, strokeWidth=2)
    .encode(
        x=alt.X(
            "rolling_km_per_week_daily_distance:Q",
            title="Daily Distance (km)",
            scale=alt.Scale(zero=False),
        ),
        y=alt.Y(
            "rolling_pace_sec_per_km:Q",
            title="Pace (mm:ss)",
            scale=alt.Scale(
                reverse=True,
                zero=False,
                domain=(min(pace_ticks_values), max(pace_ticks_values)),
            ),
            axis=alt.Axis(
                values=pace_ticks_values,
                labelExpr="datum.value > 0 ? timeFormat(datum.value * 1000, '%M:%S') : ''",
            ),
        ),
        tooltip=tooltip,
        order="date",
        color=alt.Color(
            "Legend:N",
            legend=alt.Legend(title=None),
            scale=alt.Scale(domain=["Tanda Progression line"], range=["#87f94c"]),
        ),
    )
    .properties(width=800, height=500, title="Pace and daily distance")
    .interactive()
)


current_form = (
    alt.Chart(
        daily_df.loc[start_date:last_date].reset_index().sort_values("date").tail(1)
    )
    .mark_point(filled=True, size=70)
    .encode(
        x=alt.X(
            "rolling_km_per_week_daily_distance:Q",
            title="Daily Distance (km)",
            scale=alt.Scale(zero=False),
        ),
        y=alt.Y(
            "rolling_pace_sec_per_km:Q",
            title="Pace (mm:ss)",
            scale=alt.Scale(
                reverse=True,
                zero=False,
                domain=(min(pace_ticks_values), max(pace_ticks_values)),
            ),
            axis=alt.Axis(
                values=pace_ticks_values,
                labelExpr="datum.value > 0 ? timeFormat(datum.value * 1000, '%M:%S') : ''",
            ),
        ),
        color=alt.value("#142ef5"),
        tooltip=tooltip,
    )
    .properties(width=800, height=500, title="Pace and daily distance")
    .interactive()
)

(tanda_progression + current_form).show()

alt.LayerChart(...)

In [25]:
(marathon_times + daily_line + tanda_progression + current_form).properties(
    title="Tanda and marathon pace",
).interactive().show()

alt.LayerChart(...)

In [31]:
df

,distance_meters,time_seconds
start_date,,
2023-12-27 08:33:55+00:00,6005.7,1974.0
2023-12-28 09:06:12+00:00,2540.0,991.0
2023-12-29 15:34:36+00:00,5202.3,1802.0
2023-12-30 21:35:08+00:00,10140.7,3353.0
2024-01-01 13:29:04+00:00,5000.0,1802.0
...,...,...
2024-06-18 05:17:05+00:00,5088.1,1631.0
2024-06-19 16:43:53+00:00,5479.0,1715.0
2024-06-21 14:40:47+00:00,7318.3,2346.0


In [30]:
running_activities[0]

Activity(bound_client=<stravalib.client.Client object at 0x11b74a090>, id=10443295124, achievement_count=0, athlete=Athlete(bound_client=None, id=44717295, city=None, country=None, created_at=None, firstname=None, lastname=None, premium=None, profile=None, profile_medium=None, resource_state=1, sex=None, state=None, summit=None, updated_at=None, bikes=None, clubs=None, follower_count=None, friend_count=None, ftp=None, measurement_preference=None, shoes=None, weight=None, is_authenticated=None, athlete_type=None, friend=None, follower=None, approve_followers=None, badge_type_id=None, mutual_friend_count=None, date_preference=None, email=None, super_user=None, email_language=None, max_heartrate=None, username=None, description=None, instagram_username=None, offer_in_app_payment=None, global_privacy=None, receive_newsletter=None, email_kom_lost=None, dateofbirth=None, facebook_sharing_enabled=None, profile_original=None, premium_expiration_date=None, email_send_follower_notices=None, plan